In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from time import time
from datetime import date, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
pd.set_option('display.max_columns', 500)

## 1 预处理

In [3]:
# train_data_path = "../data/train.csv"
# df_train = pd.read_csv(
#     train_data_path, usecols=[1, 2, 3, 4, 5], dtype={'onpromotion': bool}, 
#     converters={'unit_sales': lambda x: np.log1p(float(x)) if float(x) > 0 else 0}, 
#     parse_dates=['date'], skiprows=range(1, 66458909)
# )
# df_2017 = df_train[df_train['date'] >= '2017-01-01']

df_2017 = pd.read_csv("../data/train-2017.csv", parse_dates=['date'])

df_test = pd.read_csv(
    "../data/test.csv", # usecols=[0, 1, 2, 3, 4], 
    dtype={'onpromotion': bool}, parse_dates=['date']
).set_index(['store_nbr', 'item_nbr', 'date'])

items = pd.read_csv("../data/items.csv").set_index('item_nbr')

stores = pd.read_csv("../data/stores.csv").set_index("store_nbr")

In [4]:
encoder = LabelEncoder()

items['family'] = encoder.fit_transform(items['family'].values)

stores['city'] = encoder.fit_transform(stores['city'].values)
stores['state'] = encoder.fit_transform(stores['state'].values)
stores['type'] = encoder.fit_transform(stores['type'].values)

每个店铺每件商品每天的促销情况

In [5]:
promo_2017_train = df_2017.set_index(
    ['store_nbr', 'item_nbr', 'date'])[['onpromotion']].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

promo_2017_test = df_test[['onpromotion']].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

del promo_2017_train, promo_2017_test
gc.collect()

14

每个店铺每件商品每天的销量

In [6]:
df_2017 = df_2017.set_index(
    ['store_nbr', 'item_nbr', 'date'])[['unit_sales']].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [7]:
items = items.reindex(df_2017.index.get_level_values(1))
stores = stores.reindex(df_2017.index.get_level_values(0))

每件商品每天的销量

In [8]:
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()

每件商品每天的促销情况

In [9]:
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

每个种类的商品在每个店铺每天的销量

In [10]:
df_2017_store_class = df_2017.reset_index()
df_2017_store_class['class'] = items['class'].values
df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

每个种类的商品在每个店铺每天的促销情况

In [11]:
promo_2017_store_class = promo_2017.reset_index()
promo_2017_store_class['class'] = items['class'].values
promo_2017_store_class_index = promo_2017_store_class[['class', 'store_nbr']]
promo_2017_store_class = promo_2017_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

## 2 特征工程（准备数据集）

In [12]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [13]:
def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    # 促销天数特征（6个特征）
    X = {
        'promo_14_2017': get_timespan(promo_df, t2017, 14, 14).sum(axis=1).values,  # 前14天中每个店铺商品的促销天数
        'promo_60_2017': get_timespan(promo_df, t2017, 60, 60).sum(axis=1).values,  # 前60天中每个店铺商品的促销天数
        'promo_140_2017': get_timespan(promo_df, t2017, 140, 140).sum(axis=1).values,  # 前140天中每个店铺商品的促销天数
        'promo_3_2017_aft': get_timespan(promo_df, t2017 + timedelta(days=16), 15, 3).sum(axis=1).values,  # 后3天中每个店铺商品的促销天数
        'promo_7_2017_aft': get_timespan(promo_df, t2017 + timedelta(days=16), 15, 7).sum(axis=1).values,  # 后7天中每个店铺商品的促销天数
        'promo_14_2017_aft': get_timespan(promo_df, t2017 + timedelta(days=16), 15, 14).sum(axis=1).values  # 后14天中每个店铺商品的促销天数
    }
    
    # 促销销量和正常销量特征（24个特征）
    for i in [3, 7, 14, 30, 60, 140]:
        tmp1 = get_timespan(df, t2017, i, i)  # 前i天每个店铺商品的销量
        tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1  # 前i天每个店铺商品的促销情况（是否促销）
        
        X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values  # 前i天每个店铺商品平均促销销量
        X['has_promo_mean_%s_decay' % i] = (tmp1 * tmp2.replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i天每个店铺商品促销销量和（带衰减）
        
        X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values  # 前i天每个店铺商品平均正常销量
        X['no_promo_mean_%s_decay' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i天每个店铺商品正常销量和（带衰减）
        
    # 销量统计特征（42个特征）
    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)  # 前i天每个店铺商品的销量
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i天每个店铺商品销量的平均一阶差分
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i天每个店铺商品销量的和（带衰减）
        X['mean_%s' % i] = tmp.mean(axis=1).values  # 前i天每个店铺商品销量的均值
        X['median_%s' % i] = tmp.median(axis=1).values  # 前i天每个店铺商品销量的中位数
        X['min_%s' % i] = tmp.min(axis=1).values  # 前i天每个店铺商品销量的最小值
        X['max_%s' % i] = tmp.max(axis=1).values  # 前i天每个店铺商品销量的最大值
        X['std_%s' % i] = tmp.std(axis=1).values  # 前i天每个店铺商品销量的标准差
    
    # 销量统计特征2（42个特征）
    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017 + timedelta(days=-7), i, i)  # 7天前的前i天每个店铺商品的销量
        X['diff_%s_mean_2' % i] = tmp.diff(axis=1).mean(axis=1).values  # 7天前的前i天每个店铺商品销量的平均一阶差分
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 7天前的前i天每个店铺商品销量的和（带衰减）
        X['mean_%s_2' % i] = tmp.mean(axis=1).values  # 7天前的前i天每个店铺商品销量的平均值
        X['median_%s_2' % i] = tmp.median(axis=1).values  # 7天前的前i天每个店铺商品销量的中位数
        X['min_%s_2' % i] = tmp.min(axis=1).values  # 7天前的前i天每个店铺商品销量的最小值
        X['max_%s_2' % i] = tmp.max(axis=1).values  # 7天前的前i天每个店铺商品销量的最大值
        X['std_%s_2' % i] = tmp.std(axis=1).values  # 7天前的前i天每个店铺商品销量的标准差
        
    # 有销量和有促销的天数特征（30个特征）
    for i in [7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)  # 前i天每个店铺商品的销量
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i天每个店铺商品有销量的天数
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i天每个店铺商品距离上一次有销量的天数
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i天每个店铺商品距离第一次有销量的天数
        
        tmp = get_timespan(promo_df, t2017, i, i)  # 前i天每个店铺商品的促销情况
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i天每个店铺商品有促销的天数
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i天每个店铺商品距离上一次有促销的天数
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i天每个店铺商品距离第一次有促销的天数
        
    # 后15天的促销情况特征（3个特征）
    tmp = get_timespan(promo_df, t2017 + timedelta(days=16), 15, 15)  # 后15天每个店铺商品的促销情况
    X['has_promo_days_in_after_15_days'] = (tmp > 0).sum(axis=1).values  # 后15天每个店铺商品有促销的天数
    X['last_has_promo_day_in_after_15_days'] = ((tmp > 0) * np.arange(1, 16)).max(axis=1).values  # 后15天每个店铺商品距离最后一次有促销的天数
    X['first_has_promo_day_in_after_15_days'] = 16 - ((tmp > 0) * np.arange(15, 0, -1)).max(axis=1).values  # 后15天每个店铺商品距离最近一次有促销的天数
    
    # 前15天的销量（15个特征）
    for i in range(1, 16):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()
    
    # 前4（20）周每天的平均销量（14个特征）
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        
    # 前16天到后15天每天的促销情况（32个特征）
    for i in range(-16, 16):
        X['promo_{}'.format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)
        
    X = pd.DataFrame(X)
    
    if is_train:
        y = df[pd.date_range(t2017, periods=16)].values
        return X, y
    
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    
    return X

准备训练集

In [14]:
print("[INFO] Preparing training data...")

t2017 = date(2017, 6, 14)
num_days = 6
X_l, y_l = [], []
for i in range(num_days):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)
    
    X_tmp2 = prepare_dataset(df_2017_item, promo_2017_item, t2017 + delta, is_train=False, name_prefix='item')
    X_tmp2.index = df_2017_item.index
    X_tmp2 = X_tmp2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)
    
    X_tmp3 = prepare_dataset(df_2017_store_class, promo_2017_store_class, t2017 + delta, is_train=False, name_prefix='store_class')
    X_tmp3.index = df_2017_store_class.index
    X_tmp3 = X_tmp3.reindex(df_2017_store_class_index).reset_index(drop=True)
    
    X_tmp = pd.concat([X_tmp, X_tmp2, X_tmp3, items.reset_index(), stores.reset_index()], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
    
    del X_tmp, y_tmp, X_tmp2, X_tmp3
    gc.collect()

X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

del X_l, y_l
gc.collect()

[INFO] Preparing training data...


0

准备验证集

In [15]:
print("[INFO] Preparing validation data...")

X_val, y_val = prepare_dataset(df_2017, promo_2017, date(2017, 7, 26))

X_val2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 7, 26), is_train=False, name_prefix='item')
X_val2.index = df_2017_item.index
X_val2 = X_val2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_val3 = prepare_dataset(df_2017_store_class, promo_2017_store_class, date(2017, 7, 26), is_train=False, name_prefix='store_class')
X_val3.index = df_2017_store_class.index
X_val3 = X_val3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_val = pd.concat([X_val, X_val2, X_val3, items.reset_index(), stores.reset_index()], axis=1)

del X_val2, X_val3
gc.collect()

[INFO] Preparing validation data...


7

准备测试集

In [16]:
print("[INFO] Preparing testing data...")

X_test = prepare_dataset(df_2017, promo_2017, date(2017, 8, 16), is_train=False)

X_test2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 8, 16), is_train=False, name_prefix='item')
X_test2.index = df_2017_item.index
X_test2 = X_test2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_test3 = prepare_dataset(df_2017_store_class, promo_2017_store_class, date(2017, 8, 16), is_train=False, name_prefix='store_class')
X_test3.index = df_2017_store_class.index
X_test3 = X_test3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_test = pd.concat([X_test, X_test2, X_test3, items.reset_index(), stores.reset_index()], axis=1)

del X_test2, X_test3
gc.collect()

[INFO] Preparing testing data...


7

## 3 训练模型

In [ ]:
print("[INFO] Start training...")
t0 = time()

params = {
    'num_leaves': 80, 
    'objective': 'regression', 
    'min_data_in_leaf': 200, 
    'learning_rate': 0.02, 
    'feature_fraction': 0.8, 
    'bagging_fraction': 0.7, 
    'bagging_freq': 1, 
    'metric': 'l2', 
    'num_threads': 16
}

MAX_ROUNDS = 5000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print('=' * 50)
    print("Step %d" % (i + 1))
    print('=' * 50)
    
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i], 
        categorical_feature=cate_vars, 
        weight=pd.concat([items['perishable']] * num_days) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain, 
        categorical_feature=cate_vars, 
        weight=items['perishable'] * 0.25 + 1
    )
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS, 
        valid_sets=[dtrain, dval], early_stopping_rounds=125, 
        verbose_eval=50
    )
    print('\n'.join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance('gain')), key=lambda x: x[1], reverse=True)))
    val_pred.append(
        bst.predict(X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(
        bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("[INFO] Done in %f seconds." % (time() - t0))
print("[INFO] Training Finished! ( ^ _ ^ ) V")
    
print("Validation mse:", mean_squared_error(y_val, np.array(val_pred).transpose()))

weight = items['perishable'] * 0.25 + 1
err = (y_val - np.array(val_pred).transpose()) ** 2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print("nwrmsle = {}".format(err))

y_val = np.array(val_pred).transpose()
df_preds = pd.DataFrame(
    y_val, index=df_2017.index, 
    columns=pd.date_range('2017-07-26', periods=16)
).stack().to_frame('unit_sales')
df_preds.index.set_names(['store_nbr', 'item_nbr', 'date'], inplace=True)
df_preds['unit_sales'] = np.clip(np.expm1(df_preds['unit_sales']), 0, 1000)
df_preds.reset_index().to_csv('lgb_cv.csv', index=False)

[INFO] Start training...
Step 1


/root/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1040: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.412476	valid_1's l2: 0.403112
[100]	training's l2: 0.310644	valid_1's l2: 0.30659
[150]	training's l2: 0.291937	valid_1's l2: 0.290089
[200]	training's l2: 0.286146	valid_1's l2: 0.28588
[250]	training's l2: 0.282872	valid_1's l2: 0.283936
[300]	training's l2: 0.280484	valid_1's l2: 0.282787
[350]	training's l2: 0.278714	valid_1's l2: 0.282126
[400]	training's l2: 0.277237	valid_1's l2: 0.281708
[450]	training's l2: 0.275873	valid_1's l2: 0.28136
[500]	training's l2: 0.274668	valid_1's l2: 0.281121
[550]	training's l2: 0.273503	valid_1's l2: 0.280921
[600]	training's l2: 0.272394	valid_1's l2: 0.280742
[650]	training's l2: 0.271399	valid_1's l2: 0.280651
[700]	training's l2: 0.270376	valid_1's l2: 0.280516
[750]	training's l2: 0.269436	valid_1's l2: 0.280426
[800]	training's l2: 0.268507	valid_1's l2: 0.280332
[850]	training's l2: 0.267602	valid_1's l2: 0.280292
[900]	training's l2: 0.266748	valid_1's 

Step 2
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.413245	valid_1's l2: 0.418767
[100]	training's l2: 0.328991	valid_1's l2: 0.337099
[150]	training's l2: 0.313397	valid_1's l2: 0.322532
[200]	training's l2: 0.307917	valid_1's l2: 0.317753
[250]	training's l2: 0.304656	valid_1's l2: 0.315537
[300]	training's l2: 0.302219	valid_1's l2: 0.314195
[350]	training's l2: 0.300275	valid_1's l2: 0.313315
[400]	training's l2: 0.298594	valid_1's l2: 0.312795
[450]	training's l2: 0.297037	valid_1's l2: 0.312278
[500]	training's l2: 0.295658	valid_1's l2: 0.311929
[550]	training's l2: 0.294363	valid_1's l2: 0.31165
[600]	training's l2: 0.293152	valid_1's l2: 0.311369
[650]	training's l2: 0.292012	valid_1's l2: 0.311143
[700]	training's l2: 0.290909	valid_1's l2: 0.311042
[750]	training's l2: 0.289868	valid_1's l2: 0.310954
[800]	training's l2: 0.288851	valid_1's l2: 0.310892
[850]	training's l2: 0.28785	valid_1's l2: 0.310839
[900]	training's l2: 0.286903	va

Step 3
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.436126	valid_1's l2: 0.458153
[100]	training's l2: 0.335735	valid_1's l2: 0.357155
[150]	training's l2: 0.316348	valid_1's l2: 0.338489
[200]	training's l2: 0.309444	valid_1's l2: 0.333188
[250]	training's l2: 0.305322	valid_1's l2: 0.330914
[300]	training's l2: 0.302221	valid_1's l2: 0.329849
[350]	training's l2: 0.299895	valid_1's l2: 0.329109
[400]	training's l2: 0.298012	valid_1's l2: 0.328625
[450]	training's l2: 0.29632	valid_1's l2: 0.328199
[500]	training's l2: 0.294812	valid_1's l2: 0.327861
[550]	training's l2: 0.293418	valid_1's l2: 0.32758
[600]	training's l2: 0.292113	valid_1's l2: 0.327356
[650]	training's l2: 0.29086	valid_1's l2: 0.327159
[700]	training's l2: 0.289684	valid_1's l2: 0.327045
[750]	training's l2: 0.288577	valid_1's l2: 0.32696
[800]	training's l2: 0.287531	valid_1's l2: 0.326885
[850]	training's l2: 0.286463	valid_1's l2: 0.326739
[900]	training's l2: 0.285446	vali

...

Step 12
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.517231	valid_1's l2: 0.51133
[100]	training's l2: 0.396001	valid_1's l2: 0.399963
[150]	training's l2: 0.37009	valid_1's l2: 0.379949
[200]	training's l2: 0.359747	valid_1's l2: 0.373802
[250]	training's l2: 0.352886	valid_1's l2: 0.370686
[300]	training's l2: 0.347742	valid_1's l2: 0.368729
[350]	training's l2: 0.343844	valid_1's l2: 0.367561
[400]	training's l2: 0.340463	valid_1's l2: 0.366504
[450]	training's l2: 0.337666	valid_1's l2: 0.365582
[500]	training's l2: 0.335194	valid_1's l2: 0.364968
[550]	training's l2: 0.333005	valid_1's l2: 0.364449
[600]	training's l2: 0.331009	valid_1's l2: 0.364051
[650]	training's l2: 0.329158	valid_1's l2: 0.363691
[700]	training's l2: 0.327375	valid_1's l2: 0.363333
[750]	training's l2: 0.325785	valid_1's l2: 0.36312
[800]	training's l2: 0.324268	valid_1's l2: 0.362834
[850]	training's l2: 0.322863	valid_1's l2: 0.362622
[900]	training's l2: 0.321454	va

Step 13
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.483238	valid_1's l2: 0.479778
[100]	training's l2: 0.385467	valid_1's l2: 0.38817
[150]	training's l2: 0.365	valid_1's l2: 0.372912
[200]	training's l2: 0.356765	valid_1's l2: 0.368724
[250]	training's l2: 0.351059	valid_1's l2: 0.366578
[300]	training's l2: 0.347128	valid_1's l2: 0.365104
[350]	training's l2: 0.344082	valid_1's l2: 0.364127
[400]	training's l2: 0.34142	valid_1's l2: 0.363456
[450]	training's l2: 0.339004	valid_1's l2: 0.36288
[500]	training's l2: 0.337008	valid_1's l2: 0.362513
[550]	training's l2: 0.335057	valid_1's l2: 0.36212
[600]	training's l2: 0.333283	valid_1's l2: 0.36183
[650]	training's l2: 0.331607	valid_1's l2: 0.361581
[700]	training's l2: 0.330045	valid_1's l2: 0.361304
[750]	training's l2: 0.328533	valid_1's l2: 0.361161
[800]	training's l2: 0.327143	valid_1's l2: 0.361002
[850]	training's l2: 0.32577	valid_1's l2: 0.360853
[900]	training's l2: 0.324433	valid_1'

Step 14
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.475845	valid_1's l2: 0.458194
[100]	training's l2: 0.380694	valid_1's l2: 0.37287
[150]	training's l2: 0.359905	valid_1's l2: 0.359453
[200]	training's l2: 0.351301	valid_1's l2: 0.355692
[250]	training's l2: 0.345388	valid_1's l2: 0.3539
[300]	training's l2: 0.34125	valid_1's l2: 0.352594
[350]	training's l2: 0.337897	valid_1's l2: 0.351759
[400]	training's l2: 0.335171	valid_1's l2: 0.351004
[450]	training's l2: 0.332802	valid_1's l2: 0.350503
[500]	training's l2: 0.330651	valid_1's l2: 0.350141
[550]	training's l2: 0.328664	valid_1's l2: 0.349842
[600]	training's l2: 0.326909	valid_1's l2: 0.349448
[650]	training's l2: 0.325212	valid_1's l2: 0.349151
[700]	training's l2: 0.323634	valid_1's l2: 0.348915
[750]	training's l2: 0.322121	valid_1's l2: 0.348695
[800]	training's l2: 0.320674	valid_1's l2: 0.348522
[850]	training's l2: 0.31933	valid_1's l2: 0.348411
[900]	training's l2: 0.318053	vali

Step 15
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.464803	valid_1's l2: 0.448668
[100]	training's l2: 0.364555	valid_1's l2: 0.359221
[150]	training's l2: 0.343496	valid_1's l2: 0.344708
[200]	training's l2: 0.335	valid_1's l2: 0.340637
[250]	training's l2: 0.329075	valid_1's l2: 0.338452
[300]	training's l2: 0.324751	valid_1's l2: 0.336837
[350]	training's l2: 0.321398	valid_1's l2: 0.335834
[400]	training's l2: 0.318699	valid_1's l2: 0.334878
[450]	training's l2: 0.316373	valid_1's l2: 0.334116
[500]	training's l2: 0.31431	valid_1's l2: 0.333577
[550]	training's l2: 0.312432	valid_1's l2: 0.333188
[600]	training's l2: 0.310752	valid_1's l2: 0.332848
[650]	training's l2: 0.309196	valid_1's l2: 0.332498
[700]	training's l2: 0.307732	valid_1's l2: 0.332236
[750]	training's l2: 0.306326	valid_1's l2: 0.332064
[800]	training's l2: 0.305003	valid_1's l2: 0.331851
[850]	training's l2: 0.303749	valid_1's l2: 0.331755
[900]	training's l2: 0.30254	vali

Step 16
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.452781	valid_1's l2: 0.458347
[100]	training's l2: 0.36947	valid_1's l2: 0.380155
[150]	training's l2: 0.351706	valid_1's l2: 0.366205
[200]	training's l2: 0.344209	valid_1's l2: 0.362
[250]	training's l2: 0.338901	valid_1's l2: 0.359944
[300]	training's l2: 0.334922	valid_1's l2: 0.358501
[350]	training's l2: 0.331863	valid_1's l2: 0.357537
[400]	training's l2: 0.329223	valid_1's l2: 0.356721
[450]	training's l2: 0.326961	valid_1's l2: 0.35617
[500]	training's l2: 0.324984	valid_1's l2: 0.355779
[550]	training's l2: 0.323166	valid_1's l2: 0.355377
[600]	training's l2: 0.321457	valid_1's l2: 0.355104
[650]	training's l2: 0.31987	valid_1's l2: 0.354837
[700]	training's l2: 0.318407	valid_1's l2: 0.354617
[750]	training's l2: 0.317008	valid_1's l2: 0.354438
[800]	training's l2: 0.315684	valid_1's l2: 0.354301
[850]	training's l2: 0.314448	valid_1's l2: 0.354151
[900]	training's l2: 0.313227	valid

# 4 提交结果

In [ ]:
print("[INFO] Making submission...")

y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index, 
    columns=pd.date_range('2017-08-16', periods=16)
).stack().to_frame('unit_sales')
df_preds.index.set_names(['store_nbr', 'item_nbr', 'date'], inplace=True)

In [ ]:
submission = df_test[['id']].join(df_preds, how='left').fillna(0)
submission['unit_sales'] = np.clip(np.expm1(submission['unit_sales']), 0, 1000)
submission.to_csv('lgb_sub.csv', float_format='%.4f', index=None)